In [3]:
import sympy as sp

def qutrit_operators():
    """
    Calculates the displacement and phase-point operators for a qutrit system (d=3)
    symbolically using SymPy. This ensures accurate calculations with complex
    numbers and avoids floating-point errors.

    Returns:
    - D_matrices: A dictionary containing the displacement operators D(p, q)
                   for p, q in {0, 1, 2}. Each D(p, q) is a SymPy Matrix.
    - A_matrices: A dictionary containing the phase-point operators A(p, q)
                   for p, q in {0, 1, 2}. Each A(p, q) is a SymPy Matrix.
    """

    # Define the symbolic variable omega and the matrices X, Z, and Pi
    omega = sp.exp(2 * sp.pi * sp.I / 3)  # Use SymPy's exp and I
    X = sp.Matrix([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
    Z = sp.Matrix([[1, 0, 0], [0, omega, 0], [0, 0, omega**2]])
    Pi = sp.Matrix([[1, 0, 0], [0, 0, 1], [0, 1, 0]])

    # Initialize dictionaries to store the results
    D_matrices = {}
    A_matrices = {}

    # Calculate displacement operators D(p, q)
    for p in range(3):
        for q in range(3):
            # Use SymPy's matrix exponentiation and multiplication
            D_matrices[(p, q)] = (omega**(p * q)) * (Z**p) * (X**q)

    # Calculate phase-point operators A(p, q)
    for p in range(3):
        for q in range(3):
            Dpq = D_matrices[(p, q)]
            Dpq_dag = Dpq.adjoint()  # Use SymPy's adjoint() for conjugate transpose
            A_matrices[(p, q)] = Dpq * Pi * Dpq_dag

    return D_matrices, A_matrices

def display_matrices(matrices, title_prefix):
    """
    Displays the matrices in a user-friendly format using LaTeX within a Markdown
    structure.

    Args:
    - matrices: A dictionary where keys are (p, q) tuples and values are
                SymPy matrices.
    - title_prefix: A string to prepend to the title of each matrix set
                    (e.g., "Displacement Operators" or "Phase-Point Operators").
    """
    print(f"\n## {title_prefix}\n")
    for p in range(3):
        for q in range(3):
            matrix = matrices[(p, q)]
            # Use SymPy's latex() for LaTeX representation
            latex_matrix = sp.latex(matrix)
            # Fixed the output string
            print(f"${title_prefix}({p},{q}) = {latex_matrix}$")
        print("\n")  # Add extra spacing

def check_operators(D_matrices, A_matrices):
    """
    Performs sanity checks on the calculated displacement and phase-point operators.
    """
    omega = sp.exp(2 * sp.pi * sp.I / 3)

    # Check commutation relation for displacement operators
    print("Checking commutation relations...")
    for p1 in range(3):
        for q1 in range(3):
            for p2 in range(3):
                for q2 in range(3):
                    D1 = D_matrices[(p1, q1)]
                    D2 = D_matrices[(p2, q2)]
                    p_sum = (p1 + p2) % 3
                    q_sum = (q1 + q2) % 3
                    phase_factor = omega**((q1 * p2 - p1 * q2) % 3)
                    D_sum = D_matrices[(p_sum, q_sum)]
                    
                    # Check the commutation relation: D1 * D2 = phase_factor * D_sum
                    commutation_check = sp.simplify(D1 * D2 - phase_factor * D_sum)
                    
                    # Calculate maximum error in a safer way
                    max_error = 0
                    for i in range(3):
                        for j in range(3):
                            element = commutation_check[i, j]
                            if element != 0:  # Check if the element is not symbolically zero
                                try:
                                    error = abs(complex(element))
                                    max_error = max(max_error, error)
                                except:
                                    print(f"Warning: Could not convert element {element} to complex number")
                    
                    if max_error > 1e-10:
                        print(f"Warning: D({p1},{q1}) and D({p2},{q2}) commutation error: {max_error}")
                        print(f"Expected: {phase_factor * D_sum}")
                        print(f"Got: {D1 * D2}")

    # Sanity check: Hermitian and trace-1 for phase-point operators
    print("\nChecking Hermiticity and trace of phase-point operators...")
    for key, A in A_matrices.items():
        # Check if A is Hermitian: A = A†
        hermitian_diff = sp.simplify(A - A.H)
        
        # Calculate maximum Hermiticity error in a safer way
        hermitian_error = 0
        for i in range(3):
            for j in range(3):
                element = hermitian_diff[i, j]
                if element != 0:  # Check if the element is not symbolically zero
                    try:
                        error = abs(complex(element))
                        hermitian_error = max(hermitian_error, error)
                    except:
                        print(f"Warning: Could not convert element {element} to complex number")
        
        if hermitian_error > 1e-10:
            print(f"Warning: A{key} not Hermitian, error: {hermitian_error}")
        
        # Check if trace(A) = 1
        trace_diff = sp.simplify(sp.trace(A) - 1)
        if trace_diff != 0:
            try:
                trace_error = abs(complex(trace_diff))
                if trace_error > 1e-10:
                    print(f"Warning: A{key} trace error: {trace_error}")
            except:
                print(f"Warning: Could not calculate trace error for A{key}")

    print("\nAll operator checks completed!")

if __name__ == "__main__":
    # Call the function to calculate the operators
    D_matrices, A_matrices = qutrit_operators()

    # Display the results
    display_matrices(D_matrices, "D")
    display_matrices(A_matrices, "A")

    # Perform checks
    check_operators(D_matrices, A_matrices)


## D

$D(0,0) = \left[\begin{matrix}1 & 0 & 0\\0 & 1 & 0\\0 & 0 & 1\end{matrix}\right]$
$D(0,1) = \left[\begin{matrix}0 & 0 & 1\\1 & 0 & 0\\0 & 1 & 0\end{matrix}\right]$
$D(0,2) = \left[\begin{matrix}0 & 1 & 0\\0 & 0 & 1\\1 & 0 & 0\end{matrix}\right]$


$D(1,0) = \left[\begin{matrix}1 & 0 & 0\\0 & e^{\frac{2 i \pi}{3}} & 0\\0 & 0 & e^{- \frac{2 i \pi}{3}}\end{matrix}\right]$
$D(1,1) = \left[\begin{matrix}0 & 0 & e^{\frac{2 i \pi}{3}}\\e^{- \frac{2 i \pi}{3}} & 0 & 0\\0 & 1 & 0\end{matrix}\right]$
$D(1,2) = \left[\begin{matrix}0 & e^{- \frac{2 i \pi}{3}} & 0\\0 & 0 & 1\\e^{\frac{2 i \pi}{3}} & 0 & 0\end{matrix}\right]$


$D(2,0) = \left[\begin{matrix}1 & 0 & 0\\0 & e^{- \frac{2 i \pi}{3}} & 0\\0 & 0 & e^{\frac{2 i \pi}{3}}\end{matrix}\right]$
$D(2,1) = \left[\begin{matrix}0 & 0 & e^{- \frac{2 i \pi}{3}}\\e^{\frac{2 i \pi}{3}} & 0 & 0\\0 & 1 & 0\end{matrix}\right]$
$D(2,2) = \left[\begin{matrix}0 & e^{\frac{2 i \pi}{3}} & 0\\0 & 0 & 1\\e^{- \frac{2 i \pi}{3}} & 0 & 0\end{matrix}\right]$